In [2]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import plotly.express as px

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [3]:
prediction_season = 2022

In [14]:
# Base data frames

point_spread_base = pd.read_csv('../data/adjusted_stats_point_spread_modeling_base_df.csv').dropna()
over_under_base = pd.read_csv('../data/adjusted_stats_over_under_modeling_base_df.csv').dropna()
game_outcome_base = pd.read_csv('../data/adjusted_stats_modeling_base_df.csv').dropna()

In [15]:
# Feature lists

point_spread_feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'home_spread'
]

over_under_feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'total_score_line'
]

game_outcome_feature_list =[
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home' 
    , 'pass_def_value_adjusted_home' 
    , 'rush_def_value_adjusted_home' 
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away' 
    , 'rushing_value_adjusted_away' 
    , 'pass_def_value_adjusted_away' 
    , 'rush_def_value_adjusted_away' 
    , 'special_teams_value_away'
]

In [16]:
# Get features

point_spread_features = point_spread_base[point_spread_base.season == prediction_season][point_spread_feature_list].to_numpy()
over_under_features = over_under_base[over_under_base.season == prediction_season][over_under_feature_list].to_numpy()
game_outcome_features = game_outcome_base[game_outcome_base.season == prediction_season][game_outcome_feature_list].to_numpy()

In [17]:
# Import models

point_spread_model = load('../modeling/point_spread_modeling/saved_models/ps_xg_boost_av.joblib')
over_under_model = load('../modeling/over_under_modeling/saved_models/ou_xg_boost_av.joblib')
game_outcome_model = load('../modeling/game_outcome_modeling/saved_models/random_forest_av.joblib')